In [2]:
import tensorflow as tf
import numpy as np


# to reshape (5,1) back to (5,) you can use ...

X_XOR=np.array([[0,0],[0,1],[1,0],[1,1]])
Y_XOR = np.array([0,1,1,0])


X1a = tf.placeholder(tf.float32, shape=([None,len(X_XOR)]), name="Xia")
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(X1a)
    

Tensor("Xia:0", shape=(?, 4), dtype=float32)
